### Import Libraries

In [1]:
import numpy as np
import os
import pandas as pd
import random
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset, Dataset
from transformers import ConvNextModel, ConvNextImageProcessor, ConvNextForImageClassification
from config import CFG
from datasets import load_dataset
from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import time

from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')

# os.environ['CUDA_VISIBLE_DEVICES'] = '1'

/ai/anaconda3/envs/bst/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-18 00:01:47.884963: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Load data

In [2]:
label_data = pd.read_csv(os.path.join(CFG.train_data_dir, '文件标签汇总数据.csv'))
train_csv_folder = os.path.join(CFG.train_data_dir, 'csv文件')
test_A_csv_folder = os.path.join(CFG.test_A_data_dir, 'csv文件')
train_image_folder = os.path.join(CFG.project_name, 'project/image/训练集数据')
test_A_image_folder = os.path.join(CFG.project_name, 'project/image/A榜测试集数据')
label2id = {"A": 0, "B": 1, "C": 2, "D": 3, "E": 4}
id2label = {0: "A", 1: "B", 2: "C", 3: "D", 4: "E"}

In [3]:
# 定义训练集的数据增强转换流程
train_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    # resize为448*448
    transforms.Resize(448),
    # 随机裁剪为448*448
    transforms.RandomResizedCrop(448), 
    # 随机水平翻转
    transforms.RandomHorizontalFlip(),
    # 随机垂直翻转
    transforms.RandomVerticalFlip(),
    # 随机旋转
    transforms.RandomRotation(15),
    # 随机亮度
    transforms.ColorJitter(brightness=0.5),
    # 随机对比度
    transforms.ColorJitter(contrast=0.5),
    # 中心裁剪为448*448
    transforms.CenterCrop(448),
    # 随机 affine 变换
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

# 定义验证集的数据预处理流程（无数据增强）
val_transforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize(448),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])

test_trainsforms = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize(448),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),

])

In [4]:
# 加载完整数据集（未应用转换）
full_dataset = datasets.ImageFolder(root=train_image_folder)
# 分层抽样以分割数据集
targets = [sample[1] for sample in full_dataset.samples]
train_idx, val_idx = train_test_split(
    list(range(len(targets))),
    test_size=0.2,
    random_state=42,
    stratify=targets
)

In [5]:
# 应用数据转换并创建训练集和验证集的子集
# train_dataset = full_dataset
# train_dataset.transform = train_transforms  # 应用训练集转换
train_dataset = Subset(full_dataset, train_idx)
train_dataset.dataset.transform = train_transforms  # 应用训练集转换


val_dataset = Subset(full_dataset, val_idx)
val_dataset.dataset.transform = val_transforms     # 应用验证集转换

In [6]:
# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

In [7]:
dataset_sizes = {'train': len(train_dataset), 'val': len(val_dataset)}

In [8]:
dataset_sizes

{'train': 115, 'val': 23}

### 初始化模型，定义分类头

In [9]:
processor = ConvNextImageProcessor.from_pretrained(CFG.convNext_model)
model = ConvNextForImageClassification.from_pretrained(CFG.convNext_model,
                                                       num_labels=5,
                                                       ignore_mismatched_sizes=True)
model.to('cuda:1')

Some weights of ConvNextForImageClassification were not initialized from the model checkpoint at /ai/users/bst/competition/model/convnext-large-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 1536]) in the checkpoint and torch.Size([5, 1536]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([5]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ConvNextForImageClassification(
  (convnext): ConvNextModel(
    (embeddings): ConvNextEmbeddings(
      (patch_embeddings): Conv2d(3, 192, kernel_size=(4, 4), stride=(4, 4))
      (layernorm): ConvNextLayerNorm()
    )
    (encoder): ConvNextEncoder(
      (stages): ModuleList(
        (0): ConvNextStage(
          (downsampling_layer): Identity()
          (layers): Sequential(
            (0): ConvNextLayer(
              (dwconv): Conv2d(192, 192, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=192)
              (layernorm): ConvNextLayerNorm()
              (pwconv1): Linear(in_features=192, out_features=768, bias=True)
              (act): GELUActivation()
              (pwconv2): Linear(in_features=768, out_features=192, bias=True)
              (drop_path): Identity()
            )
            (1): ConvNextLayer(
              (dwconv): Conv2d(192, 192, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=192)
              (layernorm): ConvNextLayerNorm()
     

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001,weight_decay=0.0001)

In [11]:
for epoch in range(CFG.epochs):
    print('Epoch {}/{}'.format(epoch+1, CFG.epochs))
    print('-' * 10)

    # 每个epoch都有一个训练和验证阶段
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()  # 设置模型为训练模式
            dataloader = train_loader
        else:
            model.eval()   # 设置模型为评估模式
            dataloader = val_loader

        running_loss = 0.0
        running_corrects = 0
        y_true = []
        y_pred = []
        # 迭代数据
        for inputs, labels in tqdm(dataloader):
            inputs = inputs.to('cuda:1')
            labels = labels.to('cuda:1')

            # 零参数梯度
            optimizer.zero_grad()
            # 前向
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs.logits, 1)
                loss = criterion(outputs.logits, labels)
                # 后向+仅在训练阶段进行优化
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
            # 统计f1_score
            y_true.extend(labels.cpu().numpy().tolist())
            y_pred.extend(preds.cpu().numpy().tolist())
            # 统计loss
            running_loss += loss.item() * inputs.size(0)
        # 分别计算5个label的f1_score,然后加权平均[0.1, 0.3, 0.2, 0.1, 0.3]
        score = f1_score(y_true, y_pred, average=None)
        score = np.average(score, weights=[0.1, 0.3, 0.2, 0.1, 0.3])
        # 计算loss
        epoch_loss = running_loss / dataset_sizes[phase]
        print('{} Loss: {:.4f} F1-score: {:.4f}'.format(
            phase, epoch_loss, score))
        # 计算混淆矩阵
        print(pd.crosstab(pd.Series(y_true), pd.Series(y_pred), rownames=['label'], colnames=['predict']))

Epoch 1/50
----------


  0%|          | 0/15 [00:00<?, ?it/s]

100%|██████████| 15/15 [00:22<00:00,  1.51s/it]


train Loss: 1.5474 F1-score: 0.1697
predict   0   1
label          
0        31   2
1        21  12
2        20   5
3        10   6
4         5   3


100%|██████████| 3/3 [00:02<00:00,  1.41it/s]


...

val Loss: 0.0027 F1-score: 1.0000
predict  0  1  2  3  4
label                 
0        7  0  0  0  0
1        0  6  0  0  0
2        0  0  5  0  0
3        0  0  0  3  0
4        0  0  0  0  2
Epoch 50/50
----------


100%|██████████| 15/15 [00:22<00:00,  1.48s/it]


train Loss: 0.0028 F1-score: 1.0000
predict   0   1   2   3  4
label                     
0        33   0   0   0  0
1         0  33   0   0  0
2         0   0  25   0  0
3         0   0   0  16  0
4         0   0   0   0  8


100%|██████████| 3/3 [00:01<00:00,  1.96it/s]

val Loss: 0.0027 F1-score: 1.0000
predict  0  1  2  3  4
label                 
0        7  0  0  0  0
1        0  6  0  0  0
2        0  0  5  0  0
3        0  0  0  3  0
4        0  0  0  0  2


### 预测测试集

In [12]:
class TestDataset(Dataset):
    def __init__(self, image_folder, transform=None):
        self.image_folder = image_folder
        self.transform = transform
        self.image_files = [f for f in os.listdir(
            image_folder) if f.endswith('.png')]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_folder, self.image_files[idx])
        image = Image.open(img_name)
        if self.transform:
            image = self.transform(image)

        return image, self.image_files[idx]

In [13]:
test_dataset = TestDataset(test_A_image_folder, test_trainsforms)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [14]:
model.eval()
predictions = []
file_name = []
for images,img_name in tqdm(test_loader):
    with torch.no_grad():
        images = images.to('cuda:1')  
        outputs = model(images)
        _, predicted =torch.max(outputs.logits, 1)
        predictions.extend(predicted.cpu().numpy())
        file_name.extend(list(img_name))

In [15]:
csv_file_name = []
test_A_csv_file_name = os.listdir(test_A_csv_folder)
for file in file_name:
    for j in test_A_csv_file_name:
        if file[:-4] in j:
            csv_file_name.append(j)
            break

In [16]:
label = [CFG.id2label[i] for i in predictions]

In [17]:
results_df = pd.DataFrame({'defectType': label,'fileName': csv_file_name})
results_df.to_csv('submission.csv', index=False)